In [1]:
# from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from openpyxl import Workbook
from openpyxl import load_workbook

In [2]:
def get_selected_cols_df(df, cols):
    '''
    cols : list of str 
    '''
    return df[cols]



FEDEX_CARRIER_CODE = ['FDEN','FDEN_EXP','FXFE']
FILE_PATH = ".\data\IOD_MONITORING_DETAIL_US_202061515948415074903147.csv"
df_origin = pd.read_csv(FILE_PATH)
df = df_origin[df_origin['Carrier Code'].isin(FEDEX_CARRIER_CODE)]
fedex_no = get_selected_cols_df(df, cols = ['Pro No']).values.ravel()

In [3]:

fedex_no
print("we have {} fedex tracking number".format(len(fedex_no)))

we have 1265 fedex tracking number


In [4]:
df_origin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3805 entries, 0 to 3804
Data columns (total 56 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Div                          3805 non-null   object 
 1   Load ID                      3805 non-null   int64  
 2   Stop Seq                     3805 non-null   int64  
 3   Carrier Code                 3805 non-null   object 
 4   Carrier Name                 3805 non-null   object 
 5   Service Type                 3805 non-null   object 
 6   Container Type               3805 non-null   object 
 7   Customer PO                  3805 non-null   object 
 8   Alert Type                   3805 non-null   object 
 9   Sales                        3805 non-null   float64
 10  Ship Date                    3805 non-null   object 
 11  Required Delivery Date       3805 non-null   object 
 12  Drop Appointment             427 non-null    object 
 13  IOD Report Date   

In [5]:
df_origin.Div.unique()

array(['LGEUS'], dtype=object)

In [6]:
fedex_no

array(['187887450070', '187887452349', '192470114721', ...,
       '191974238790', '191974238780', '5402058071'], dtype=object)

In [21]:
from bs4 import BeautifulSoup
import requests
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36',
'Referer': 'https://www.fedex.com/trackingCal/track',
'Host': 'www.fedex.com',
'Cookie': 'fdx_cbid=30036807161594754925222750021081; fdx_locale=en_US; fdx_redirect=en-us; cc_path=us; s_ecid=MCMID%7C91240026263101807612795946297234918876; _abck=E6BF6C129F738D2B9D1C7477355EE277~0~YAAQL5EvFwBtVC1zAQAAAwvNTgSEKQJaNIT46+OyW/Rl1EiG2zsHB8tUiwjmoECJgd2c5DwAVVZJqwRE1WyaZUW6CBEIrzjTPlAfDtIA0eWI1PZx8cmmPMA8K043L/XtLVvra0HYHOpMsVBS/T9heRtUPhD61LP9fCtzlX8ASW2c/LueOQF0WnVI6fhEBpzT7lxVey3V7rIXOIdQx785tSS1+3g9TOPU74tgm0TvJyIpOTjLhX27RoU7wo+mzSD5HKwhzKC3pckP0hjKF1EGAXRyh7NQTD4m08aQDFxo0f0BS7BbSbLuVXHDuKdgHLRb5nJkiLs4~-1~-1~-1; _gcl_au=1.1.494437621.1594754928; tracking_locale=en_US; wdpl_id=30036807161594754925222750021081_1594754941226; optimizelyEndUserId=oeu1594842710172r0.13632731098668605; Rbt=f0; s_vi=[CS]v1|2F8820BB8515E1A5-6000073AEC279C6F[CE]; AMCVS_1E22171B520E93BF0A490D44%40AdobeOrg=1; Nina-nina-fedex-session=%7B%22loginStatus%22%3A%22loggedOut%22%2C%22locale%22%3A%22en_us%22%2C%22lcstat%22%3Afalse%7D; isTablet=false; isMobile=false; isWireless=false; level=test; bm_sz=5BB46F53147E7BBCD47B4C37BC78D810~YAAQlnRiaJPVWi1zAQAAJXw3XQh6C6TL+Dl3dGbK6+qjsXocUy5liNmMW2fQizJguuxcwYMTEaQPJi4PryCjShUBBiaYYKmSQjjhl0a+GMl7F6U0KDHejOkVJ/1pT1/yNfe+CmOB7L4GDBX2Da71uMsI/OrXZy9nG/NpqU0o7SGr9ak3xGEm0cAluilt/g==; siteDC=edc; xacc=US; bm_mi=D97EA7AE59A71D64ABC1CE72F7DCD025~65MQ2A3vVx5vSgxC/yeDl2lFxkBeglbJqGXd/VYDV+/QQUfzatmIlR/gqdrWWsEWwSAKY02onSd4OA/vcruEut8C7LhCxqKU2pxgqvAYpMKxDceAVEx65KsBPspYs7bJGRjjvv4K2iArVWm3AFmGFU+u7xMDPd/VmaK90SxrIMZZt9Wf2vTLDia0rW00j72L2g/2pX5r/CSuu1ZTji9YkXa1JJu4t5ZPqRR9ioW4MA6fZnIisi6oRTFnvG1LMCfjB7vsu8qwH15V3Y4cDGMOpu+QTm9N73hyDN+PdcCe+Bk=; AMCV_1E22171B520E93BF0A490D44%40AdobeOrg=1075005958%7CMCIDTS%7C18460%7CMCMID%7C91240026263101807612795946297234918876%7CMCAAMLH-1595603246%7C7%7CMCAAMB-1595603246%7CRKhpRz8krg2tLO6pguXWp5olkAcUniQYPHaMWWgdJ3xzPWQmdj0y%7CMCOPTOUT-1595005646s%7CNONE%7CMCAID%7CNONE%7CvVersion%7C4.4.1; ak_bmsc=FC7D1B6DC6190F8D92E20B6077227DE668627496244C000030B8115F544A3924~plB1Uo1cwUBtX+dabNP9VLhCrKlX94LvhLTxprMRd21ckNlouJ/BObEiZctT172w8I19GMSknLavDO7UDweLyJMMadFhGO3nE4GwT3C8ZpdzLO1eoE9ipz9QvNk8DoBS+fKbm71445nC8d/jVdt/lrSnMQcLLrO3bDSAqWBtRHIi5JpiaYlC8aM6WFCJe0qeaisaLKFKBwqFu/o7QWnLRxJqXPzivs0pj9X6lhfyiAGt1D337A/dli+5j8pWRODwdf; check=true; s_cc=true; aemserver=Prod-c0016055.prod.cloud.fedex.com; s_sq=%5B%5BB%5D%5D; mbox=PC#24d3ece21dd041fb900d4602ebedfe62.34_0#1596209614|session#1594998445845-631570#1595001874; s_pers=%20s_nr%3D1594998447881-Repeat%7C1626534447881%3B%20s_vnum%3D1595044800891%2526vn%253D1%7C1595044800891%3B%20gpv_pageName%3Dfedex%252Fhome%7C1595000846868%3B%20s_invisit%3Dtrue%7C1595000846880%3B%20s_dfa%3Dfedexglbl%252Cfedexus%7C1595001813932%3B; s_sess=%20s_cc%3Dtrue%3B%20s_visit%3D1%3B%20SC_LINKS%3D%3B%20s_ppv%3Dfedex%252Fhome%252C30%252C30%252C1069%3B%20setLink%3D%3B; bm_sv=E1780F789453D566F93B2BD5AB6D54C1~KsLgz3VZeT0wZvPEL/kiCF0oeNBd+he095kk1yoIO4SD/EkW/29goeYQqhL7UeeDrtH+hh2myeHL6f54dZ+jCrq5sAEOVJoShD65gSI/BYO/cFBTCHuHngWfVCOxtAPkq/sTQGsWatfbW1eJChn6yRm4JLMVM1DOnpyUqwOO3cc='
} 
url_id = 'https://www.fedex.com/apps/fedextrack'
params_id = {
'action': 'track',
'tracknumbers': '906213154240,906213154170,906213154284,906213154262',
'locale':'en_US',
'cntry_code':'us'} 
req_id= requests.post(url_id,params=params_id,headers = headers)

In [23]:
req_id.text

'<!DOCTYPE html>\n\n\t\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n<html>\n\t<head>\n\t\t<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" >\n\t\t<META http-equiv="Content-Type" content="text/html; charset=UTF-8">\n\t\t<meta http-equiv="X-UA-Compatible" content="IE=edge" />\r\n<meta http-equiv="content-type" content="text/html; charset=utf-8" />\r\n<link rel="canonical" href="http://www.fedex.com/fedextrack/"/>\r\n<title></title>\r\n<meta name="apple-mobile-web-app-capable" content="yes" />\r\n<meta name="apple-mobile-web-app-status-bar-style" content="default" />\r\n<meta name="viewport" content="initial-scale=1.0; maximum-scale=1.0; user-scalable=no" />\r\n<meta content="fedex,fedex tracking,fedex shipping,freight,delivery services" name="keywords">\r\n<meta content="FedEx tracking provides unparalleled insight into when your package will be delivered. FedEx tracking also allows you to manage your shipments." name="description">\r\n\r\n<!-- 2 Meta tags needed for Site Catalyst tag

In [20]:
req_id.content
soup = BeautifulSoup(req_id.text, 'html.parser')
soup.body

<body id="fx-respond"> <!--<![endif]-->
<script>var fx_appshell = true;var fx_responsive = "true";</script>
<div id="container">
<script>var vh=0;
var fxg_header=true;
</script>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>

 ï»¿<link href="//www.fedex.com/css/legacy/main-min.css" rel="stylesheet" type="text/css"/>
<!-- HEAD SCRIPTS -->
<script>
vh=1;
var fxg_header=true;


/* @param [string]  [styleName] [filename with suffix e.g. "style.css"]
 * @param [boolean] [disabled]  [true disables style]
 */

        


var fx_device_type = "desktop";
var fxg_header = true;


var disableStyle = function(styleName, disabled) {
    var styles = document.styleSheets;
    var href = "";
try {
    for (var i = 0; i < styles.length; i++) {
    	  var re = new RegExp(".*?//.*?/", "");
  	    var cssStr =styles[i].href.replace(re,"/");
    	  var r = /[^\/]*$/;
    	  //alert('cssStr   ' + cssStr);
				var dirPath = cssStr.replace(r, '');
				//alert ('dirPath   ' + dirPath);
      

In [12]:
url_mes = f"https://www.fedex.com/apps/fedextrack/?action={params_id['action']}&tracknumbers={params_id['tracknumbers']}'&locale={params_id['locale']}&cntry_code={params_id['cntry_code']}"
req = requests.get(url_mes,headers = headers)
js = req.json()


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [13]:
req

<Response [200]>

In [11]:
soup = BeautifulSoup(page.text, 'html.parser')
soup.body

<body id="fx-respond"> <!--<![endif]-->
<script>var fx_appshell = true;var fx_responsive = "true";</script>
<div id="container">
<script>var vh=0;
var fxg_header=true;
</script>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>

 ï»¿<link href="//www.fedex.com/css/legacy/main-min.css" rel="stylesheet" type="text/css"/>
<!-- HEAD SCRIPTS -->
<script>
vh=1;
var fxg_header=true;


/* @param [string]  [styleName] [filename with suffix e.g. "style.css"]
 * @param [boolean] [disabled]  [true disables style]
 */

        


var fx_device_type = "desktop";
var fxg_header = true;


var disableStyle = function(styleName, disabled) {
    var styles = document.styleSheets;
    var href = "";
try {
    for (var i = 0; i < styles.length; i++) {
    	  var re = new RegExp(".*?//.*?/", "");
  	    var cssStr =styles[i].href.replace(re,"/");
    	  var r = /[^\/]*$/;
    	  //alert('cssStr   ' + cssStr);
				var dirPath = cssStr.replace(r, '');
				//alert ('dirPath   ' + dirPath);
      

In [9]:
print(soup.prettify())


<!DOCTYPE html>
<html>
 <head>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible">
   <meta content="text/html; charset=utf-8" http-equiv="content-type">
    <link href="http://www.fedex.com/fedextrack/" rel="canonical"/>
    <title>
    </title>
    <meta content="yes" name="apple-mobile-web-app-capable"/>
    <meta content="default" name="apple-mobile-web-app-status-bar-style"/>
    <meta content="initial-scale=1.0; maximum-scale=1.0; user-scalable=no" name="viewport"/>
    <meta content="fedex,fedex tracking,fedex shipping,freight,delivery services" name="keywords"/>
    <meta content="FedEx tracking provides unparalleled insight into when your package will be delivered. FedEx tracking also allows you to manage your shipments." name="description"/>
    <!-- 2 Meta tags needed for Site Catalyst tagging -->
    <meta content="unifiedtracking" nam

In [68]:
soup

<!DOCTYPE html>

<html>
<head>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible">
<meta content="text/html; charset=utf-8" http-equiv="content-type">
<link href="http://www.fedex.com/fedextrack/" rel="canonical"/>
<title></title>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<meta content="default" name="apple-mobile-web-app-status-bar-style"/>
<meta content="initial-scale=1.0; maximum-scale=1.0; user-scalable=no" name="viewport"/>
<meta content="fedex,fedex tracking,fedex shipping,freight,delivery services" name="keywords"/>
<meta content="FedEx tracking provides unparalleled insight into when your package will be delivered. FedEx tracking also allows you to manage your shipments." name="description"/>
<!-- 2 Meta tags needed for Site Catalyst tagging -->
<meta content="unifiedtracking" name="wsstitle">
<meta content="us/en/fedex/unified/"

In [18]:
soup.find_all("tbody",class_='redesignPageableTableTVC tank-mps__body')


[]

In [ ]:

css(".product::text").extract_first()

class FedExShipmentTracking:
    def __init__(self, excel_file_path, col_name = "col"):
        self.df = pd.read_excel(excel_file_path)
        self.shipment_ids = self.df[col_name].values
        
        
        
    def __call__(self, max_amount_track = 30):
        re
        
        for idx in range(0, len(self.df),max_amount_track):
            
            if idx + 30 <= len(self.df):
                response = self.crawler.send(self.shipment_ids[idx:idx+30])
            else:
                response = self.crawler.send(self.shipment_ids[idx:])
                
                
        return 